In [54]:
import fitsio as fio
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from astropy.coordinates import SkyCoord
import numpy as np
import matplotlib.pyplot as plt

In [55]:
mem = fio.FITS('/lsst/troxel/y1a1/lgt5_member_mcal_combined.fits')[-1] # Member/shape catalog
clus = fio.FITS('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17_lgt5_desformat_catalog.fit')[-1] # Cluster catalog
cent = fio.FITS('/lsst/troxel/y1a1/central_member_shapes.fits')[-1] # Central/shape catalog

In [56]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3) # Use standard cosmological model

In [57]:
# Slice to the members of specified cluster
def get_clus(catalog, clus_id):
    clus_mems = catalog.read()[catalog.read()['MEM_MATCH_ID']==clus_id]
    
    return clus_mems

In [58]:
# Calculate comoving distance
def co_distance(z):
    co_distance = cosmo.comoving_distance(z)
    
    return co_distance

In [59]:
# Calculate angular diameter distance
def ad_distance(z):
    ad_distance = co_distance(z) / (z + np.ones(len(z)))
    
    return ad_distance

In [60]:
#ad_distance(get_clus(mem, 201)['Z'])

In [61]:
# Calculate position angle
def pos_ang(e1, e2):
    alpha = np.degrees(np.arctan(e2 / e1) / 2) * u.deg
    
    return alpha

In [62]:
#pos_ang(get_clus(cent, 201)['e1'], get_clus(cent, 201)['e2'])

In [63]:
# Calculate angular separation
def separation(clus_id):
    cent_pos = SkyCoord(np.ones(len(get_clus(mem, clus_id)['RA']))*get_clus(cent, clus_id)['RA']*u.deg, np.ones(len(get_clus(mem, clus_id)['DEC']))*get_clus(cent, clus_id)['DEC']*u.deg, frame='icrs')
    sat_pos = SkyCoord(get_clus(mem, clus_id)['RA']*u.deg, get_clus(mem, clus_id)['DEC']*u.deg, frame='icrs')
    sep = cent_pos.separation(sat_pos)
    
    return sep

In [64]:
#separation(201)

In [65]:
# Calculate 3D distance
def distance_3d(clus_id):
    cent_distance = ad_distance(get_clus(cent, clus_id)['Z'])
    sat_distance = ad_distance(get_clus(mem, clus_id)['Z'])
    cent_pos = SkyCoord(np.ones(len(get_clus(mem, clus_id)['RA']))*get_clus(cent, clus_id)['RA']*u.deg, np.ones(len(get_clus(mem, clus_id)['DEC']))*get_clus(cent, clus_id)['DEC']*u.deg, distance=np.ones(len(get_clus(mem, clus_id)['Z']))*cent_distance, frame='icrs')
    sat_pos = SkyCoord(get_clus(mem, clus_id)['RA']*u.deg, get_clus(mem, clus_id)['DEC']*u.deg, distance=sat_distance, frame='icrs')
    sep_3d = cent_pos.separation_3d(sat_pos)
    
    return sep_3d

In [66]:
#distance_3d(201)

In [67]:
# Approximate 3D distance
def approx_distance(clus_id):
    ang_sep = separation(clus_id)
    cent_distance = ad_distance(get_clus(cent, clus_id)['Z'])
    approx_distance = np.radians(ang_sep) * cent_distance
    
    return approx_distance

In [68]:
#approx_distance(201)

In [69]:
# Calculate satellite angular position with respect to central
def sat_ang_pos(clus_id):
    cent_pos = SkyCoord(np.ones(len(get_clus(mem, clus_id)['RA']))*get_clus(cent, clus_id)['RA']*u.deg, np.ones(len(get_clus(mem, clus_id)['DEC']))*get_clus(cent, clus_id)['DEC']*u.deg, frame='icrs')
    sat_pos = SkyCoord(get_clus(mem, clus_id)['RA']*u.deg, get_clus(mem, clus_id)['DEC']*u.deg, frame='icrs')
    sat_ang = cent_pos.position_angle(sat_pos).to(u.deg)
    
    return sat_ang

In [70]:
#sat_ang_pos(201)

In [71]:
def cent_gal_ang(clus_id):
    cent_pos_ang = np.ones(len(sat_ang_pos(clus_id))) * pos_ang(get_clus(cent, clus_id)['e1'], get_clus(cent, clus_id)['e2'])
    sat_pos_ang = sat_ang_pos(clus_id)
    ang_diff = sat_pos_ang - cent_pos_ang
    
    return ang_diff

In [72]:
#cent_gal_ang(201)